<a href="https://colab.research.google.com/github/leksandre/-crab-for-web2ipr_book_shop/blob/main/gDriveToMega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# //!pip3 install mega
# //!pip install mega.py==0.8.5
!pip install mega.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotly 5.24.1 requires tenacity>=6.2.0, but you have tenacity 5.1.5 which is incompatible.


In [4]:
from mega import Mega
import os

In [5]:
from re import S
def human_size(bytes, units=[' bytes','KB','MB','GB','TB', 'PB', 'EB']):
    """ Returns a human readable string representation of bytes """
    return str(bytes) + units[0] if bytes < 1024 else human_size(bytes>>10, units[1:])

from pathlib import Path
from os.path import isdir

class UploadMega():
    def __init__(self, id, password):
        mega = Mega()
        self.mega = mega.login(id, password)
    def upload_single_file(self, path):
        print("uploading :", path)
        self.mega.upload(path)
        print("uploaded :", path)

    def compare_dir(self, path, newFolder = ''):
        files = os.listdir(path)
        j = 0
        for i in files:
            if isdir(i):
              print("\n\ngo to DIR :", i)
              l = self.mega.find(i)
              if l:
                print(' exist DIR ', newFolder+ "/"+i , l)
                self.compare_dir(os.path.join(path, i), newFolder+'/'+i)
              else:
                print("have to create DIR:", i)
            else:
              # print("go to file :", newFolder, " / ",i)
              l = self.mega.find(i)
              # print('check exist file ', newFolder+ "/"+i , l)
              if l:
                # print('already exist file', i)
                j = j+1

                hasErr = False
                s = (os.path.getsize(os.path.join(path, i)))
                if l[1]['s'] != s:
                  print('!!!!!----wrong file size 1', i, l[1]['s'], "<>", s )
                  print(human_size(l[1]['s']))
                  hasErr = True

                s = (os.stat(os.path.join(path, i)).st_size )
                if l[1]['s'] != s:
                  print('!!!!!----wrong file size 2', i, l[1]['s'], "<>", s )
                  print(human_size(l[1]['s']))
                  hasErr = True

                s = (Path(os.path.join(path, i)).stat().st_size)
                if l[1]['s'] != s:
                  print('!!!!!----wrong file size 3', i, l[1]['s'], "<>", s )
                  print(human_size(l[1]['s']))
                  hasErr = True

                if hasErr:
                  m.delete(l[0])
                  print("----------replace----------------have to upload file:", newFolder+ "/"+i)
                  Folder = self.mega.find(newFolder)
                  try:
                    self.mega.upload(os.path.join(path, i), Folder[0])
                  except Exception:
                    print("Exception -------replace-------- ", newFolder, " / ",i)

                # print(j ,l,' exist file ', newFolder+ "/"+i , l)

                # print(l[1]['s'])
                # print(human_size(l[1]['s']))

                # print(os.path.getsize(os.path.join(path, i)))
                # print(os.stat(os.path.join(path, i)).st_size )
                # print(Path(os.path.join(path, i)).stat().st_size)

                pass
              else:
                print("--------------------------have to upload file:", newFolder+ "/"+i)
                Folder = self.mega.find(newFolder)
                try:
                  self.mega.upload(os.path.join(path, i), Folder[0])
                except Exception:
                  print("Exception --------------- ", newFolder, " / ",i)

        print(j,' end path ',path)

    def upload_directory(self, path, newFolder = ''):
        files = os.listdir(path)
        for i in files:
            if isdir(i):
              if i == 'media2':
                continue
              print("uploading DIR :", i)
              # if len(newFolder)>0:
              l = self.mega.find(i)
              print(' exist DIR ', newFolder+ "/"+i , l)
              if l:
                print('already exist')
              else:
                self.mega.create_folder(newFolder+'/'+i)
              self.upload_directory(os.path.join(path, i), newFolder+'/'+i)
              print("uploaded DIR:", i)
            else:
              print("uploading :", newFolder, " / ",i)
              l = self.mega.find(i)
              print(' exist file ', newFolder+ "/"+i , l)
              if l:
                print('already exist')
                continue
              Folder = self.mega.find(newFolder)
              try:
                self.mega.upload(os.path.join(path, i), Folder[0])
              except Exception:
                print("Exception --------------- ", newFolder, " / ",i)
              print("uploaded :", i)

    def upload_files_by_extension(self, path, extension):
        files = os.listdir(path)
        for i in files:
            if i.split(".")[-1] == extension:
                print("uploading :", i)
                self.mega.upload(os.path.join(path, i))
                print("uploaded :", i)

In [6]:
m = UploadMega();

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/My Drive/mega/
%ls

/content/drive/My Drive/mega
desktop.ini  media/  media2/  media3/  Rubbish/


In [9]:
import os, os.path

# simple version for working with CWD
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

# path joining version for other paths
DIR = '/content/drive/My Drive/mega/media'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

# path joining version for other paths
DIR = '/content/drive/My Drive/mega/media2'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

# path joining version for other paths
DIR = '/content/drive/My Drive/mega/media3'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

1
50
132
535


In [10]:
# %ls media2

In [11]:
 m.compare_dir('/content/drive/My Drive/mega/','mega')



go to DIR : media2
 exist DIR  mega/media2 ('aFwElCbB', {'h': 'aFwElCbB', 'p': '2V5kGABQ', 'u': '9l6toXbouWk', 't': 1, 'a': {'n': 'media2'}, 'k': (1559134340, 1741178111, 3267902549, 2434543638), 'ts': 1727428052, 'key': (1559134340, 1741178111, 3267902549, 2434543638)})
132  end path  /content/drive/My Drive/mega/media2


go to DIR : Rubbish
 exist DIR  mega/Rubbish ('iIgDWDDY', {'h': 'iIgDWDDY', 'p': '2V5kGABQ', 'u': '9l6toXbouWk', 't': 1, 'a': {'n': 'Rubbish'}, 'k': (1468886512, 875765807, 2895259420, 3528253591), 'ts': 1727436051, 'key': (1468886512, 875765807, 2895259420, 3528253591)})
--------------------------have to upload file: mega/Rubbish/tmp
Exception ---------------  mega/Rubbish  /  tmp
0  end path  /content/drive/My Drive/mega/Rubbish


go to DIR : media
 exist DIR  mega/media ('OMpETBaZ', {'h': 'OMpETBaZ', 'p': '2V5kGABQ', 'u': '9l6toXbouWk', 't': 1, 'a': {'n': 'media'}, 'k': (897617684, 1489617407, 3296464001, 1513981383), 'ts': 1727436459, 'key': (897617684, 1489617

In [12]:
# m.upload_directory('/content/drive/My Drive/mega/','mega')

In [13]:
print('success')

success
